# Models

## distilbert-base-uncased-finetuned-sst-2-english

distilbert-base-uncased-finetuned-sst-2-english
Binary sentiment (POSITIVE / NEGATIVE)
https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english

## SamLowe/roberta-base-go_emotions	

SamLowe/roberta-base-go_emotions	
28 fine-grained emotions (admiration → worry)
https://huggingface.co/SamLowe/roberta-base-go_emotions

## unitary/unbiased-toxic-roberta	

unitary/unbiased-toxic-roberta	

Toxicity & six sub-types (toxic, severe_toxic, obscene, etc.)


https://huggingface.co/unitary/unbiased-toxic-roberta

## Hate-speech-CNERG/dehatebert-mono-english

Hate-speech-CNERG/dehatebert-mono-english
Hate / non-hate
https://huggingface.co/Hate-speech-CNERG/dehatebert-mono-english

## classla/multilingual-IPTC-news-topic-classifier

classla/multilingual-IPTC-news-topic-classifier

205 IPTC NewsCodes topics (e.g., crime, culture, health)

https://huggingface.co/classla/multilingual-IPTC-news-topic-classifier

# Prerequisites

In [1]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [2]:
# pip install transformers datasets evaluate bitsandbytes accelerate peft

Check that CUDA is enabled

In [3]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available in PyTorch: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

PyTorch version: 2.7.0+cu128
CUDA available in PyTorch: True
CUDA version: 12.8
GPU: NVIDIA GeForce RTX 3060


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Chosen model

distilbert-base-uncased-finetuned-sst-2-english

In [6]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          pipeline)

ds = load_dataset("go_emotions", "simplified")

tok_r = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
mdl_r = AutoModelForSequenceClassification.from_pretrained(
            "SamLowe/roberta-base-go_emotions").to("cuda")

clf = pipeline("text-classification",
               model=mdl_r, tokenizer=tok_r,
               device=0, batch_size=32, top_k=None)

def predict_split(split):
    out = clf(split["text"])
    return [{d["label"]: d["score"] for d in row} for row in out]

valid_logits  = predict_split(ds["validation"])
test_logits   = predict_split(ds["test"])
ds["validation"] = ds["validation"].add_column("roberta_logits", valid_logits)
ds["test"]       = ds["test"].add_column("roberta_logits",  test_logits)


c:\nextcloud\Studia - PW\semestr 6\warsztaty_badawcze\llm-workshop\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


In [72]:
print(ds['train'][0])

{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej', 'roberta_logits': {'admiration': 0.16123420000076294, 'amusement': 0.0019722634460777044, 'anger': 0.0015387308085337281, 'annoyance': 0.004159059375524521, 'approval': 0.23341229557991028, 'caring': 0.0017751016421243548, 'confusion': 0.0018894611857831478, 'curiosity': 0.0012016475666314363, 'desire': 0.0021674567833542824, 'disappointment': 0.00200709979981184, 'disapproval': 0.0038329351227730513, 'disgust': 0.0014345556264743209, 'embarrassment': 0.0004492918960750103, 'excitement': 0.023490943014621735, 'fear': 0.0005178069695830345, 'gratitude': 0.0010927062248811126, 'grief': 0.0003230682050343603, 'joy': 0.08386469632387161, 'love': 0.4214438796043396, 'nervousness': 0.0003635652828961611, 'neutral': 0.10018648207187653, 'optimism': 0.0028960644267499447, 'pride': 0.003201066516339779, 'realization': 0.013179345056414604, 'relief': 0.002387171844020486, 'remorse': 0.0003059

# Creating the train data set

In [8]:
import pandas as pd
import numpy as np
from datasets import Dataset
from tqdm import tqdm


In [9]:
emotion_labels = ds["validation"].features["labels"].feature.names

In [10]:
def create_emotion_probabilities_df(dataset_split):
    texts = []
    emotion_probs = {emotion: [] for emotion in emotion_labels}
    
    for text, roberta_logits in zip(dataset_split["text"], dataset_split["roberta_logits"]):
        texts.append(text)
        
        for emotion in emotion_labels:
            prob = roberta_logits.get(emotion, 0.0)
            emotion_probs[emotion].append(prob)
    
    df_data = {'input_text': texts}
    df_data.update(emotion_probs)
    
    df = pd.DataFrame(df_data)
    return df

def predict_with_dataset_optimized(dataset_split, batch_size=128):
    texts = dataset_split["text"]
    temp_dataset = Dataset.from_dict({"text": texts})
    
    def predict_batch(batch):
        try:
            outputs = clf(batch["text"], truncation=True, max_length=512)
            predictions = [{d["label"]: d["score"] for d in row} for row in outputs]
            return {"roberta_logits": predictions}
        except Exception as e:
            empty_preds = [{label: 0.0 for label in emotion_labels} for _ in range(len(batch["text"]))]
            return {"roberta_logits": empty_preds}
    
    result_dataset = temp_dataset.map(
        predict_batch,
        batched=True,
        batch_size=batch_size,
        remove_columns=["text"]
    )
    
    predictions = result_dataset["roberta_logits"]
    return predictions

def predict_with_dataset_progress(dataset_split, batch_size=128):
    texts = dataset_split["text"]
    total_samples = len(texts)
    chunk_size = 5000
    all_predictions = []
    
    for start_idx in tqdm(range(0, total_samples, chunk_size)):
        end_idx = min(start_idx + chunk_size, total_samples)
        chunk_texts = texts[start_idx:end_idx]
        
        chunk_dataset = Dataset.from_dict({"text": chunk_texts})
        
        def predict_batch(batch):
            outputs = clf(batch["text"], truncation=True, max_length=512)
            return {"roberta_logits": [{d["label"]: d["score"] for d in row} for row in outputs]}
        
        chunk_result = chunk_dataset.map(
            predict_batch,
            batched=True,
            batch_size=batch_size,
            remove_columns=["text"]
        )
        
        all_predictions.extend(chunk_result["roberta_logits"])
    
    return all_predictions

if 'roberta_logits' not in ds['train'].column_names:
    try:
        train_logits = predict_with_dataset_optimized(ds["train"], batch_size=128)
    except Exception as e:
        train_logits = predict_with_dataset_progress(ds["train"], batch_size=64)
    
    ds["train"] = ds["train"].add_column("roberta_logits", train_logits)

train_df = create_emotion_probabilities_df(ds["train"])
validation_df = create_emotion_probabilities_df(ds["validation"])
test_df = create_emotion_probabilities_df(ds["test"])

Map: 100%|██████████| 43410/43410 [01:00<00:00, 719.66 examples/s]


In [23]:
train_df = create_emotion_probabilities_df(ds["train"])

In [24]:
train_df

,input_text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,0.161234,0.001972,0.001539,0.004159,0.233412,0.001775,0.001889,0.001202,0.002167,...,0.421444,0.000364,0.002896,0.003201,0.013179,0.002387,0.000306,0.000815,0.001866,0.100186
1,"Now if he does off himself, everyone will thin...",0.000930,0.134094,0.003719,0.026139,0.011867,0.001842,0.001364,0.000668,0.001538,...,0.000506,0.000549,0.006830,0.000483,0.013376,0.000889,0.000497,0.001741,0.000749,0.840025
2,WHY THE FUCK IS BAYLESS ISOING,0.005290,0.004081,0.781201,0.122706,0.003216,0.001910,0.008621,0.012958,0.001135,...,0.002823,0.000530,0.002151,0.000427,0.002942,0.000247,0.000637,0.004392,0.005214,0.105964
3,To make her feel threatened,0.001722,0.001781,0.006446,0.019079,0.008645,0.030293,0.001861,0.001075,0.004182,...,0.000735,0.035958,0.008934,0.000745,0.008453,0.002275,0.001704,0.028242,0.001346,0.596796
4,Dirty Southern Wankers,0.004310,0.002896,0.484007,0.412610,0.006365,0.001364,0.001875,0.001695,0.001533,...,0.001446,0.000364,0.001642,0.000530,0.003438,0.000259,0.000684,0.003494,0.001129,0.081161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,0.026941,0.007407,0.003022,0.004451,0.044037,0.006570,0.002259,0.001861,0.009586,...,0.879727,0.001147,0.008405,0.001667,0.006569,0.003192,0.001235,0.004176,0.002155,0.018019
43406,Always thought that was funny but is it a refe...,0.003609,0.134665,0.002364,0.011986,0.016143,0.002777,0.742798,0.473192,0.002103,...,0.005241,0.004450,0.009182,0.000210,0.046299,0.001058,0.002461,0.003309,0.015203,0.048970
43407,What are you talking about? Anything bad that ...,0.001845,0.000994,0.022686,0.173971,0.018671,0.005762,0.177501,0.349683,0.001740,...,0.001105,0.004448,0.005316,0.000227,0.014244,0.000642,0.007382,0.026744,0.003809,0.205243
43408,"More like a baptism, with sexy results!",0.218485,0.005374,0.001724,0.003316,0.053343,0.000947,0.001491,0.002307,0.003576,...,0.014900,0.001004,0.004851,0.010119,0.007214,0.002234,0.000232,0.000925,0.018660,0.136118


In [13]:
train_df.columns

Index(['input_text', 'admiration', 'amusement', 'anger', 'annoyance',
       'approval', 'caring', 'confusion', 'curiosity', 'desire',
       'disappointment', 'disapproval', 'disgust', 'embarrassment',
       'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
       'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
       'sadness', 'surprise', 'neutral'],
      dtype='object')

In [25]:
emotion_cols = train_df.columns.drop("input_text")
emotion_values = train_df[emotion_cols].values
emotion_values

array([[1.61234200e-01, 1.97226345e-03, 1.53873081e-03, ...,
        8.14727973e-04, 1.86631945e-03, 1.00186482e-01],
       [9.29759583e-04, 1.34094074e-01, 3.71863181e-03, ...,
        1.74149009e-03, 7.48839928e-04, 8.40024650e-01],
       [5.29006822e-03, 4.08113888e-03, 7.81201065e-01, ...,
        4.39203624e-03, 5.21371607e-03, 1.05963692e-01],
       ...,
       [1.84532825e-03, 9.93798836e-04, 2.26864778e-02, ...,
        2.67438944e-02, 3.80882295e-03, 2.05242589e-01],
       [2.18485102e-01, 5.37432684e-03, 1.72380323e-03, ...,
        9.24984226e-04, 1.86597891e-02, 1.36117727e-01],
       [2.28378270e-02, 7.70451725e-02, 2.98157847e-03, ...,
        3.63864470e-03, 1.18638168e-03, 3.58057767e-02]],
      shape=(43410, 28))

In [26]:

top_3_indices = np.argpartition(emotion_values, -3, axis=1)[:, -3:]
top_3_sorted = np.take_along_axis(top_3_indices,
                                  np.argsort(np.take_along_axis(emotion_values, top_3_indices, axis=1), axis=1)[:, ::-1],
                                  axis=1)
train_df['emotion_1'] = [emotion_cols[idx] for idx in top_3_sorted[:, 0]]
train_df['emotion_2'] = [emotion_cols[idx] for idx in top_3_sorted[:, 1]]
train_df['emotion_3'] = [emotion_cols[idx] for idx in top_3_sorted[:, 2]]

train_df['emotion_1_value'] = emotion_values[np.arange(len(emotion_values)), top_3_sorted[:, 0]]
train_df['emotion_2_value'] = emotion_values[np.arange(len(emotion_values)), top_3_sorted[:, 1]]
train_df['emotion_3_value'] = emotion_values[np.arange(len(emotion_values)), top_3_sorted[:, 2]]

In [27]:
train_df

,input_text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,remorse,sadness,surprise,neutral,emotion_1,emotion_2,emotion_3,emotion_1_value,emotion_2_value,emotion_3_value
0,My favourite food is anything I didn't have to...,0.161234,0.001972,0.001539,0.004159,0.233412,0.001775,0.001889,0.001202,0.002167,...,0.000306,0.000815,0.001866,0.100186,love,approval,admiration,0.421444,0.233412,0.161234
1,"Now if he does off himself, everyone will thin...",0.000930,0.134094,0.003719,0.026139,0.011867,0.001842,0.001364,0.000668,0.001538,...,0.000497,0.001741,0.000749,0.840025,neutral,amusement,annoyance,0.840025,0.134094,0.026139
2,WHY THE FUCK IS BAYLESS ISOING,0.005290,0.004081,0.781201,0.122706,0.003216,0.001910,0.008621,0.012958,0.001135,...,0.000637,0.004392,0.005214,0.105964,anger,annoyance,neutral,0.781201,0.122706,0.105964
3,To make her feel threatened,0.001722,0.001781,0.006446,0.019079,0.008645,0.030293,0.001861,0.001075,0.004182,...,0.001704,0.028242,0.001346,0.596796,neutral,fear,nervousness,0.596796,0.246723,0.035958
4,Dirty Southern Wankers,0.004310,0.002896,0.484007,0.412610,0.006365,0.001364,0.001875,0.001695,0.001533,...,0.000684,0.003494,0.001129,0.081161,anger,annoyance,disgust,0.484007,0.412610,0.108915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,0.026941,0.007407,0.003022,0.004451,0.044037,0.006570,0.002259,0.001861,0.009586,...,0.001235,0.004176,0.002155,0.018019,love,joy,approval,0.879727,0.427309,0.044037
43406,Always thought that was funny but is it a refe...,0.003609,0.134665,0.002364,0.011986,0.016143,0.002777,0.742798,0.473192,0.002103,...,0.002461,0.003309,0.015203,0.048970,confusion,curiosity,amusement,0.742798,0.473192,0.134665
43407,What are you talking about? Anything bad that ...,0.001845,0.000994,0.022686,0.173971,0.018671,0.005762,0.177501,0.349683,0.001740,...,0.007382,0.026744,0.003809,0.205243,curiosity,neutral,confusion,0.349683,0.205243,0.177501
43408,"More like a baptism, with sexy results!",0.218485,0.005374,0.001724,0.003316,0.053343,0.000947,0.001491,0.002307,0.003576,...,0.000232,0.000925,0.018660,0.136118,excitement,admiration,neutral,0.427780,0.218485,0.136118


## Clearly seperated

In [ ]:
train_df_clearly_seperated = train_df.copy()

mask = (train_df_clearly_seperated['emotion_1_value'] > 0.6) & \
       (train_df_clearly_seperated['emotion_1_value'] >= 2 * train_df_clearly_seperated['emotion_2_value'])

train_df_clearly_seperated = train_df_clearly_seperated[mask]

print(f"Original dataset size: {len(train_df)}")
print(f"Filtered dataset size: {len(train_df_clearly_seperated)}")
print(f"Percentage retained: {len(train_df_clearly_seperated) / len(train_df) * 100:.2f}%")
train_df_clearly_seperated

Original dataset size: 43410
Filtered dataset size: 28243
Percentage retained: 65.06%


,input_text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,remorse,sadness,surprise,neutral,emotion_1,emotion_2,emotion_3,emotion_1_value,emotion_2_value,emotion_3_value
1,"Now if he does off himself, everyone will thin...",0.000930,0.134094,0.003719,0.026139,0.011867,0.001842,0.001364,0.000668,0.001538,...,0.000497,0.001741,0.000749,0.840025,neutral,amusement,annoyance,0.840025,0.134094,0.026139
2,WHY THE FUCK IS BAYLESS ISOING,0.005290,0.004081,0.781201,0.122706,0.003216,0.001910,0.008621,0.012958,0.001135,...,0.000637,0.004392,0.005214,0.105964,anger,annoyance,neutral,0.781201,0.122706,0.105964
6,Yes I heard abt the f bombs! That has to be wh...,0.010199,0.013303,0.001450,0.003774,0.057711,0.014411,0.011663,0.042843,0.010112,...,0.002114,0.002764,0.006621,0.018133,gratitude,excitement,approval,0.891421,0.215594,0.057711
7,We need more boards and to create a bit more s...,0.013856,0.002689,0.001615,0.008555,0.036555,0.006715,0.001840,0.005422,0.722661,...,0.001109,0.002065,0.001195,0.149801,desire,neutral,optimism,0.722661,0.149801,0.137451
9,It might be linked to the trust factor of your...,0.001255,0.000969,0.001125,0.005800,0.027530,0.003146,0.013570,0.002922,0.001720,...,0.000610,0.001687,0.000640,0.951158,neutral,approval,realization,0.951158,0.027530,0.019378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43398,I thought [NAME] was dope. Never understood th...,0.003202,0.003038,0.004255,0.017288,0.020478,0.001960,0.893626,0.015791,0.001829,...,0.003401,0.003748,0.004740,0.158409,confusion,neutral,realization,0.893626,0.158409,0.060395
43399,kinda obviously satire op,0.003236,0.009259,0.002834,0.012978,0.027070,0.000671,0.001370,0.001130,0.000704,...,0.000257,0.001283,0.000898,0.940813,neutral,approval,annoyance,0.940813,0.027070,0.012978
43402,I don't know if I'm comfortable with a religio...,0.007504,0.002620,0.005351,0.021828,0.053092,0.005964,0.903833,0.019431,0.004354,...,0.005500,0.005227,0.004465,0.064450,confusion,disapproval,neutral,0.903833,0.112747,0.064450
43405,Added you mate well I’ve just got the bow and ...,0.026941,0.007407,0.003022,0.004451,0.044037,0.006570,0.002259,0.001861,0.009586,...,0.001235,0.004176,0.002155,0.018019,love,joy,approval,0.879727,0.427309,0.044037


In [69]:
input_output_df = train_df_clearly_seperated.copy()[["input_text", "emotion_1"]].rename(columns={"input_text": "input", "emotion_1": "target"}).reset_index(drop=True)
input_output_df

,input,target
0,"Now if he does off himself, everyone will thin...",neutral
1,WHY THE FUCK IS BAYLESS ISOING,anger
2,Yes I heard abt the f bombs! That has to be wh...,gratitude
3,We need more boards and to create a bit more s...,desire
4,It might be linked to the trust factor of your...,neutral
...,...,...
28238,I thought [NAME] was dope. Never understood th...,confusion
28239,kinda obviously satire op,neutral
28240,I don't know if I'm comfortable with a religio...,confusion
28241,Added you mate well I’ve just got the bow and ...,love


## All

In [37]:
input_output_df = train_df.copy()[["input_text", "emotion_1"]].rename(columns={"input_text": "input", "emotion_1": "target"}).reset_index(drop=True)
input_output_df

,input,target
0,My favourite food is anything I didn't have to...,love
1,"Now if he does off himself, everyone will thin...",neutral
2,WHY THE FUCK IS BAYLESS ISOING,anger
3,To make her feel threatened,neutral
4,Dirty Southern Wankers,anger
...,...,...
43405,Added you mate well I’ve just got the bow and ...,love
43406,Always thought that was funny but is it a refe...,confusion
43407,What are you talking about? Anything bad that ...,curiosity
43408,"More like a baptism, with sexy results!",excitement


# Prompt engineering

In [70]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.05s/it]


In [ ]:
def get_model_prediction(input_text, prompt_template=None, max_new_tokens=10, do_sample=False, temperature=0.7):
    if prompt_template is None:
        prompt = f"What single concept best explains the output? Input: {input_text}"
    else:
        prompt = prompt_template.format(input_text=input_text)
   
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
   
    generation_kwargs = {
        "max_new_tokens": max_new_tokens,
        "do_sample": do_sample,
        "pad_token_id": tokenizer.eos_token_id
    }
    
    if do_sample:
        generation_kwargs["temperature"] = temperature
   
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_kwargs
        )
   
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

In [83]:
first_10 = input_output_df.head(10).copy()
prompts = {
    "concept": "What single concept best explains the output? Input: {input_text}",
    "emotion": "What emotion does this text convey? Text: {input_text}\nEmotion:",
    "topic": "Identify the main topic of this text in one word: {input_text}\nTopic:",
    "summary": "Summarize this in one word: {input_text}\nWord:"
}

for prompt_name, prompt_template in prompts.items():
    first_10[f'pred_{prompt_name}'] = first_10['input'].apply(
        lambda x: get_model_prediction(
            x, 
            prompt_template=prompt_template,
            max_new_tokens=10,
            do_sample=prompt_name != "concept",
            temperature=0.1
        )
    )

first_10

,input,target,pred_concept,pred_emotion,pred_topic,pred_summary
0,"Now if he does off himself, everyone will thin...",neutral,. Output: Conspiracy. The concept of,The text conveys a sense of irony,### Response:\nDeception,Hypocrisy\n\nSummary: Hypoc
1,WHY THE FUCK IS BAYLESS ISOING,anger,? Output:,Anger\n\nInput:\nText: WH,Document:\n\nI'm not,Reason\n\nSummary: Bayless is leaving
2,Yes I heard abt the f bombs! That has to be wh...,gratitude,����������,The text conveys a sense of relief and,Bombings\n\nInput:\n\nI',Bombastic\n\nSummary: Bombastic
3,We need more boards and to create a bit more s...,desire,Output:\n\nSpace optimization,Frustration\n\nText: I'm sorry,Space\n\nInput:\n\nDocument:,Expansion\n\nInstruction 2 (M
4,It might be linked to the trust factor of your...,neutral,Input: The trust factor of your friend,Suspicion\n\nInput:\nText:,Trust\n\nText:\n\nI'm,Trust\n\nSummary: The statement suggests that the
5,"Aww... she'll probably come around eventually,...",amusement,Options:\n (A). Empath,The text conveys a sense of empath,Relationships\n\nInput:\n\nDocument,Patience\n\nDocument:\n\nThe document
6,Hello everyone. Im from Toronto as well. Can c...,neutral,Output:\n\nThe single concept that,The text conveys a sense of openness,Toronto\n\nInput:\n\nHello everyone.,Toronto\n\nSummary: Toronto\n\nSummary:
7,R/sleeptrain Might be time for some sleep trai...,caring,# Answer\nThe single concept that best,The text conveys a sense of determination,Sleep Training\n\nInput:\n\nText,Sleep\n\nInstruction 2 (More
8,Thank you friend,gratitude,for your help. Output: Gratitude.,Gratitude\n\nText: I'm,Friendship\n\nText:\n\nI',Gratitude\n\nSummary: Gratitude
9,Fucking coward.,anger,Output: coward.\n\nInput: I,The text conveys anger.\n\nInput,Insult\n\nInput:\n\nOutput:,Cowardice.\n\nInput:


In [88]:
grouped_samples = input_output_df.groupby('target').head(10).copy()

In [ ]:
concept_prompt = "What single concept best explains the output? Input: {input_text}"

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)

result_df

,prompt,output,real_output
0,"Now if he does off himself, everyone will thin...",. Output: Conspiracy. The concept of,neutral
1,WHY THE FUCK IS BAYLESS ISOING,? Output:,anger
2,Yes I heard abt the f bombs! That has to be wh...,����������,gratitude
3,We need more boards and to create a bit more s...,Output:\n\nSpace optimization,desire
4,It might be linked to the trust factor of your...,Input: The trust factor of your friend,neutral
...,...,...,...
245,I’m so nervous!,I have to give a presentation in front of my,nervousness
246,Seeing people overtaking on the right makes me...,Output: Anxiety.\n\nInput:,nervousness
247,I'm so nervous!,I have to give a presentation in front of my,nervousness
248,Osaka making me nervous,. Output: Fear.\n\nInput:,nervousness


In [90]:
concept_prompt = "What single emotion best explains the what the author is feeling? Input: {input_text}"

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)

result_df

,prompt,output,real_output
0,"Now if he does off himself, everyone will thin...",.\n\nOutput:The author is feeling fr,neutral
1,WHY THE FUCK IS BAYLESS ISOING,ALL THE BABIES?\n\nOutput:,anger
2,Yes I heard abt the f bombs! That has to be wh...,����������,gratitude
3,We need more boards and to create a bit more s...,Output:Frustration\n\nInput:,desire
4,It might be linked to the trust factor of your...,**Solution 1:**,neutral
...,...,...,...
245,I’m so nervous!,I can’t believe I have to present my,nervousness
246,Seeing people overtaking on the right makes me...,Output:The single emotion that best,nervousness
247,I'm so nervous!,"I've been practicing for weeks, but",nervousness
248,Osaka making me nervous,. Output: Anxiety.\n\nInput,nervousness


In [94]:
grouped_samples = input_output_df.groupby('target').head(10).copy()

concept_prompt = f"What single emotion out of the following: {', '.join(emotion_cols)} best explains what the author is feeling? Input: {{input_text}}"

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)
result_df

,prompt,output,real_output
0,"Now if he does off himself, everyone will thin...",.\n\nOutput:The author is most likely,neutral
1,WHY THE FUCK IS BAYLESS ISOING,ALL THE MEN IN THE WORLD?,anger
2,Yes I heard abt the f bombs! That has to be wh...,Output:The author is feeling excitement.,gratitude
3,We need more boards and to create a bit more s...,The author' explanation of the need for,desire
4,It might be linked to the trust factor of your...,**Solution:**The emotion,neutral
...,...,...,...
245,I’m so nervous!,I’ve never been in a situation like this,nervousness
246,Seeing people overtaking on the right makes me...,The single emotion that best explains what,nervousness
247,I'm so nervous!,"I've been practicing for weeks, but",nervousness
248,Osaka making me nervous,.\n\nOutput:The author is feeling nerv,nervousness


In [ ]:
def get_model_prediction(input_text):
    prompt = f"What single concept best explains the output? Input: {input_text}"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

first_10 = input_output_df.head(10).copy()
first_10['model_prediction'] = first_10['input'].apply(get_model_prediction)
first_10[['input', 'target', 'model_prediction']]

,input,target,model_prediction
0,"Now if he does off himself, everyone will thin...",neutral,. Output: Conspiracy. The concept of
1,WHY THE FUCK IS BAYLESS ISOING,anger,? Output:
2,Yes I heard abt the f bombs! That has to be wh...,gratitude,����������
3,We need more boards and to create a bit more s...,desire,Output:\n\nSpace optimization
4,It might be linked to the trust factor of your...,neutral,Input: The trust factor of your friend
5,"Aww... she'll probably come around eventually,...",amusement,Options:\n (A). Empath
6,Hello everyone. Im from Toronto as well. Can c...,neutral,Output:\n\nThe single concept that
7,R/sleeptrain Might be time for some sleep trai...,caring,# Answer\nThe single concept that best
8,Thank you friend,gratitude,for your help. Output: Gratitude.
9,Fucking coward.,anger,Output: coward.\n\nInput: I


In [115]:
grouped_samples = input_output_df.groupby('target').head(10).copy()

concept_prompt = f"What single emotion out of the following: {', '.join(emotion_cols)} best explains what the author is feeling? Input: {{input_text}} Finish the sentence: Output:The author is feeling: "

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)
result_df

,prompt,output,real_output
0,"Now if he does off himself, everyone will thin...",Disgust.\n\n\nThe,neutral
1,WHY THE FUCK IS BAYLESS ISOING,Disgust.,anger
2,Yes I heard abt the f bombs! That has to be wh...,Explanation:\nThe author',gratitude
3,We need more boards and to create a bit more s...,1. Anger\n2. An,desire
4,It might be linked to the trust factor of your...,Disappointment.,neutral
...,...,...,...
245,I’m so nervous!,Nervousness.,nervousness
246,Seeing people overtaking on the right makes me...,,nervousness
247,I'm so nervous!,Nervousness.,nervousness
248,Osaka making me nervous,Nervousness.,nervousness


# Evaluation

In [116]:
def check_prediction(row):
    prediction = row['output'].lower()
    target = row['real_output'].lower()
    if target in prediction:
        return 1
    
    for emotion in emotion_cols:
        if emotion.lower() in prediction and emotion.lower() != target:
            return -1
    return 0

result_df['prediction_true'] = result_df.apply(check_prediction, axis=1)
result_df

,prompt,output,real_output,prediction_true
0,"Now if he does off himself, everyone will thin...",Disgust.\n\n\nThe,neutral,-1
1,WHY THE FUCK IS BAYLESS ISOING,Disgust.,anger,-1
2,Yes I heard abt the f bombs! That has to be wh...,Explanation:\nThe author',gratitude,0
3,We need more boards and to create a bit more s...,1. Anger\n2. An,desire,-1
4,It might be linked to the trust factor of your...,Disappointment.,neutral,-1
...,...,...,...,...
245,I’m so nervous!,Nervousness.,nervousness,1
246,Seeing people overtaking on the right makes me...,,nervousness,0
247,I'm so nervous!,Nervousness.,nervousness,1
248,Osaka making me nervous,Nervousness.,nervousness,1


In [117]:
result_df[result_df["prediction_true"] != 0]

,prompt,output,real_output,prediction_true
0,"Now if he does off himself, everyone will thin...",Disgust.\n\n\nThe,neutral,-1
1,WHY THE FUCK IS BAYLESS ISOING,Disgust.,anger,-1
3,We need more boards and to create a bit more s...,1. Anger\n2. An,desire,-1
4,It might be linked to the trust factor of your...,Disappointment.,neutral,-1
6,Hello everyone. Im from Toronto as well. Can c...,Admiration.\n\n\nThe,neutral,-1
...,...,...,...,...
244,"As someone with sinus and breathing problems, ...",Disgust.\n\n\nThe,nervousness,-1
245,I’m so nervous!,Nervousness.,nervousness,1
247,I'm so nervous!,Nervousness.,nervousness,1
248,Osaka making me nervous,Nervousness.,nervousness,1


In [119]:
print(f"Correct predictions: {(result_df[result_df["prediction_true"] != 0]["prediction_true"]==1).sum()}")
print(f"Incorrect predictions: {(result_df[result_df["prediction_true"] != 0]["prediction_true"]==-1).sum()}")

Correct predictions: 61
Incorrect predictions: 119


Dużo lepiej niż losowe :)